In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")

HSP_T (generic function with 1 method)

In [2]:
using QuantumOptics

In [3]:
# Operator form of SP with Theta Phases 

function OP_HSP_T(Nx, Ny, alpha, Tx, Ty)
    
    N = Nx*Ny
    
    H_T = HSP_T(Nx, Ny, alpha, Tx,Ty)
    
    basis = NLevelBasis(N) #SP Basis
    
    H = SparseOperator(basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

OP_HSP_T (generic function with 1 method)

In [5]:
using LinearAlgebra

# check operator form
eigenenergies(dense(OP_HSP_T(5, 5, 1/2, 3, 3))) == eigvals(HSP_T(5, 5, 1/2, 3, 3))

true

In [6]:
# MB Hamiltonian with Twisted-Angle

function MB_HSP_T(Nx, Ny, alpha, Tx, Ty, PN)
    
    N = Nx*Ny
    
    basis = NLevelBasis(N)
    states = fermionstates(basis, [PN])
    basis_mb = ManyBodyBasis(basis, states)
    
    H = manybodyoperator(basis_mb, OP_HSP_T(Nx, Ny, alpha, Tx, Ty))
    
    return H
end

eigenenergies(dense(MB_HSP_T(3, 3, 1/3, 2, 2, 2)))

36-element Vector{Float64}:
 -4.247407710954486
 -4.247407710954484
 -4.247407710954483
 -3.55192691581006
 -3.551926915810057
 -3.5519269158100566
 -2.1998220466366223
 -2.199822046636622
 -2.199822046636621
 -2.0402999780699203
 -2.0402999780699194
 -2.0402999780699194
 -0.4648260879066999
  ⋮
  1.6757463443171412
  1.675746344317143
  1.675746344317145
  1.9881605117701484
  1.9881605117701497
  1.9881605117701506
  3.8160389973446867
  3.816038997344687
  3.8160389973446938
  4.754502722512921
  4.754502722512924
  4.754502722512928

In [12]:
# Twisted Angle Space for Calculating Real Space Chern Numbers

Tsize=20
dx=2*pi/Tsize
dy=dx
Tx=collect(range(start=0, stop=2*pi, step=dx))
Ty=collect(range(start=0, stop=2*pi, step=dy))

21-element Vector{Float64}:
 0.0
 0.3141592653589793
 0.6283185307179586
 0.9424777960769379
 1.2566370614359172
 1.5707963267948966
 1.8849555921538759
 2.199114857512855
 2.5132741228718345
 2.827433388230814
 3.141592653589793
 3.4557519189487724
 3.7699111843077517
 4.084070449666731
 4.39822971502571
 4.71238898038469
 5.026548245743669
 5.340707511102648
 5.654866776461628
 5.969026041820607
 6.283185307179586

In [16]:
function Chern_Nums(Nx, Ny, alpha, PN, Tx, Ty, n1, n2)
    
    Sum=0
    
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            w1, v1 = eigen(dense(MB_HSP_T(Nx, Ny, alpha, Tx[tx], Ty[ty], PN)).data)
            v1 = v1[:,n1:n2]
            #------------------------------------
            w2, v2 = eigen(dense(MB_HSP_T(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty], PN)).data)
            v2 = v2[:,n1:n2]
            #------------------------------------
            w3, v3 = eigen(dense(MB_HSP_T(Nx, Ny, alpha, Tx[tx], Ty[ty]+dy, PN)).data)
            v3 = v3[:,n1:n2]
            #------------------------------------
            w4, v4 = eigen(dense(MB_HSP_T(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty]+dy, PN)).data)
            v4 = v4[:,n1:n2]
            #----------LINK VARIABLES------------
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
        end
    end
    
    return 1/(2*pi*1im)*Sum
end

#Nx=Ny=3 iken:
#n1=1;n2=12
#n3=13;n4=24
#n5=25;n6=36

Chern_Nums(3, 3, 1/3, 2, Tx, Ty, 1, 11)

-0.1702518160790502 + 1.4724495787715159e-16im